*3.1* Aggregate Quries

In [2]:
import sqlite3
import pandas as pd

In [3]:
# Connect to the SQLite database

conn =  sqlite3.connect("../database/db.sqlite3")

# Create a cursor object
cursor = conn.cursor()



In [3]:
#1. Student Analytics
# -------------------

# Count total number of students
count_query = f''' 
                SELECT COUNT(*) as total_students 
                FROM students;
                '''
count_result = pd.read_sql(count_query, conn)
print(count_result)

# use of 'as'

# In SQL, the "AS" keyword is used to rename a column or table in the result set of a 
# query. This is particularly useful for making the output more readable or for 
# simplifying column names in complex queries. 

# run the code without ' as total_student ' to see the difference 

   total_students
0               8


In [17]:
# Count students by department_id
sCount_query = f'''
                SELECT department_id, COUNT(*) as student_per_department
                FROM students
                GROUP BY department_id                
 '''
sCount_result = pd.read_sql(sCount_query,conn)
print(sCount_result)


   department_id  student_per_department
0              1                       4
1              2                       2
2              3                       2


> Now, you're probably thinking, 'What's this **GROUP BY** business all about?' Well, wonder no more! We're about to dive in and explore the wonderful world of **GROUP BY**!

>>The GROUP BY clause in SQL is used to group rows that have the same values in specified columns into aggregated data. It is often used in conjunction with aggregate functions like *COUNT()*, *SUM()*, *AVG()*, *MIN()*, and *MAX()* to perform calculations on each group of data.

**Rules**:
   - Columns in SELECT must be either:
     * Included in GROUP BY clause
     * Used with aggregate functions
   - Groups rows with matching values
   - Applied before HAVING clause

In [4]:
# Get Age demographics of students 
age_query = f''' 
            SELECT 
                strftime('%Y', date_of_birth) as birth_year,
                COUNT(*) as number_of_students
            FROM students
            GROUP BY birth_year
            ORDER BY birth_year;
            '''

age_query_result = pd.read_sql(age_query,conn)
print(age_query_result)

  birth_year  number_of_students
0       2000                   3
1       2001                   2
2       2002                   2
3       2003                   1


> Worried about *strftime('%Y')*

>>The purpose of this line is to extract the year from a date of birth and format it as a string in the format '%Y', which stands for a four-digit year  

In [5]:
enroll_query= f''' 
                SELECT 
                    strftime('%Y', enrollment_date) as year,
                    COUNT(*) as yearly_enrollments
                FROM enrollments
                GROUP BY year
                ORDER BY year;
            ''' 
enroll_query_result = pd.read_sql(enroll_query,conn)
print(enroll_query_result)

   year  yearly_enrollments
0  2023                   6
1  2024                   2
2  2025                   1


> ORDER BY? Where did THAT come from?!

>> Ah, ORDER BY, the ultimate party crasher - shows up uninvited, but rearranges everything anyway!

>> The **ORDER BY** clause in SQL is used to sort the result set returned by a query based on one or more columns. In your example, the **ORDER BY** year clause is used to sort the result set by the *year* column in *ascending* order.

>> If you wanted to sort in descending order, you would use **ORDER BY** year **DESC**.

In [10]:
#Salary statistics
stat_query = f'''
          SELECT 
            MIN(salary) as lowest_salary,
            MAX(salary) as highest_salary,
            AVG(salary) as average_salary
        FROM role;  
 '''

stat_query_result = pd.read_sql(stat_query,conn)
print(stat_query_result)

# I am hoping this cell is self-explanatory 

   lowest_salary  highest_salary  average_salary
0          30000           80000         58000.0


In [12]:
#Find students with multiple phone numbers
phone_query = f'''
                SELECT 
                    student_id,
                    COUNT(*) as phone_count
                FROM student_phones
                GROUP BY student_id
                HAVING phone_count > 1;
            '''
phone_query_result = pd.read_sql(phone_query,conn)
print(phone_query_result)

   student_id  phone_count
0           1            5
1           2            4
2           3            2
3           4            2
4           5            3
5           6            2
6           7            2


> So, Let`s talk about our last clause **Having**
>> The **HAVING** clause was added to SQL because the **WHERE** keyword cannot be used with aggregate functions. 
* bouncer! right ?

>> Let me Explain --
>> - SQL queries are executed in a specific logical order. A simplified version of this order is:
>>      - SELECT: Select the columns to be included in the result.
>>      - FROM and JOIN: Determine the source tables and how they're combined.
>>      - WHERE: Filter individual rows based on specified conditions.
>>      - GROUP BY: Group rows with the same values in specified columns.
>>      - HAVING: Filter groups of rows based on specified conditions.
>>      - ORDER BY: Sort the results.

>> - The Problem with WHERE and Aggregate Functions:
>>      - The WHERE clause operates on individual rows before any grouping or aggregation occurs.
>>      - Aggregate functions ( i.e. COUNT(), SUM(), AVG(), MAX(), MIN() ) calculate values based on groups of rows after the WHERE clause has been applied.
>>      - Therefore, you can't use WHERE to filter results based on the output of an aggregate function because that output doesn't exist yet when the WHERE clause is evaluated.

>Example: Table : **Sales**

| P_id     | quantity |
| -------- | -------  |
| 1        | 10       |
| 2        | 20       |
| 3        | 30       |

> SQL:
>> SELECT product_id, SUM(quantity)
>>
>> FROM sales
>> 
>> WHERE SUM(quantity) > 25
>>
>> GROUP BY product_id;
>>

> Why This Fails:

- *WHERE* Clause Evaluation:
    -  The *WHERE* clause is processed before the GROUP BY and SUM() operations.

    - At this stage, the database is looking at individual rows. It doesn't know the total quantity for each product_id yet.

    - *SUM(quantity)* is an aggregate function that needs to be calculated on groups of rows. Since the rows have not been grouped yet, SUM(quantity) cannot be evaluated.

    - Therefore, the WHERE *SUM(quantity) > 25* condition is invalid. The database doesn't know what SUM(quantity) is at this point.


> SQL:
>>
>>SELECT product_id, SUM(quantity)
>>
>>FROM sales
>>
>>GROUP BY product_id
>>
>>HAVING SUM(quantity) > 25;
>> 
> Why This Works:

- GROUP BY and SUM():
    - The GROUP BY product_id clause groups the rows by product_id.
    - The SUM(quantity) function is then calculated for each group, giving us the total quantity sold for each product_id.
- *HAVING* Clause Evaluation:
    - The HAVING SUM(quantity) > 25 clause is evaluated after the SUM() calculations.
    - It filters the groups where the total quantity is greater than 25.

In [15]:
#Department Distribution
dist_query = f''' 
            SELECT 
                department_id,
                COUNT(*) as student_count
            FROM students
            GROUP BY department_id
            HAVING student_count > 1
            ORDER BY student_count DESC;
            '''
dist_query_result = pd.read_sql(dist_query,conn)
print(dist_query_result)

   department_id  student_count
0              1              4
1              3              2
2              2              2


In [4]:
# Calculate the total salary for all roles
sum_query = f'''
            SELECT 
                SUM(salary) as total_salary
            FROM role;
            '''
sum_query_result = pd.read_sql(sum_query, conn)
print(sum_query_result)

   total_salary
0        290000
